In [31]:
import seaborn as sns
df = sns.load_dataset('tips')
df.head(2)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3


In [32]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [34]:
# converting catagorical features into int
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [40]:
df['time'].nunique()

2

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    int32   
 6   size        244 non-null    int64   
dtypes: category(3), float64(2), int32(1), int64(1)
memory usage: 8.0 KB


In [53]:
# independent & dependent features
X =df.drop(labels=['time'], axis=1)
y = df['time']

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [55]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((183, 6), (61, 6), (183,), (61,))

In [56]:
categorical_cols = ['sex', 'smoker','day']
numerical_cols = ['total_bill', 'tip','size']

In [57]:
from sklearn.impute import SimpleImputer  ##handeling missing values
from sklearn.preprocessing import StandardScaler  ##feature scaling
from sklearn.pipeline import Pipeline  ## automating
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder ##catagorical missing values handeling

In [58]:
##  Feature engeering automation
num_pipeline = Pipeline(
        steps= [
            ('imputer', SimpleImputer(strategy='median')), ## missing values
            ('scaler', StandardScaler())
            ]
            )
cat_pipeline = Pipeline(
                steps=[
                     ('imputer', SimpleImputer(strategy='most_frequent')),
                    ('onehotencoder', OneHotEncoder())
                    ]

                     )

In [59]:
preprocessor = ColumnTransformer([
                ('num_pipeline', num_pipeline, numerical_cols),
                ('cat_pipeline', cat_pipeline, categorical_cols)])

In [60]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [89]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB

In [93]:
# automate the model training process
models= {
    'Random Forest':RandomForestClassifier(),
    'Decision Tree':DecisionTreeClassifier(),
    'SVC' : SVC(),
    'Naive Bayes(G)': GaussianNB(),
    'Naive Bayes(B)' : BernoulliNB()
}

In [94]:
from sklearn.metrics import accuracy_score

In [95]:
def evaluate_model(X_train,y_train, X_test,y_test,models):
    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        # model training
        model.fit(X_train, y_train)
        
        
        #predict test data
        y_test_pred = model.predict(X_test)
        
        
        #accuracy
        test_model_score = accuracy_score(y_test,y_test_pred)
        
        report[list(models.keys())[i]] = test_model_score
        
        
    return report

In [96]:
evaluate_model(X_train,y_train, X_test,y_test,models)

{'Random Forest': 0.9672131147540983,
 'Decision Tree': 0.9508196721311475,
 'SVC': 0.9672131147540983,
 'Naive Bayes(G)': 0.9508196721311475,
 'Naive Bayes(B)': 0.9836065573770492}

In [103]:
classifier = RandomForestClassifier()
params = {
    'max_depth':[3,5,10,None],
              'n_estimators':[100,200,300],
               'criterion':['gini','entropy']
    
}

In [104]:
from sklearn.model_selection import RandomizedSearchCV
cv = RandomizedSearchCV(classifier,param_distributions=params,cv=5,scoring='accuracy',verbose=3)
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.946 total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.973 total time=   0.1s
[CV 3/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.973 total time=   0.1s
[CV 4/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.917 total time=   0.1s
[CV 5/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.944 total time=   0.1s
[CV 1/5] END criterion=gini, max_depth=10, n_estimators=300;, score=0.919 total time=   0.6s
[CV 2/5] END criterion=gini, max_depth=10, n_estimators=300;, score=0.946 total time=   0.6s
[CV 3/5] END criterion=gini, max_depth=10, n_estimators=300;, score=1.000 total time=   0.5s
[CV 4/5] END criterion=gini, max_depth=10, n_estimators=300;, score=0.944 total time=   0.5s
[CV 5/5] END criterion=gini, max_depth=10, n_estimators=300;, score=0.917 total time=   0.6s

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [105]:
cv.best_params_

{'n_estimators': 100, 'max_depth': 5, 'criterion': 'entropy'}

In [106]:
cv.best_score_

0.9615615615615616